In [2]:
import custom_models
#python packages
from PIL import Image
from tqdm.notebook import tqdm
#from tqdm import tqdm
import gc
import datetime
import os
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from skimage import io
#torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
#torchvision
import torchvision
from torchvision import datasets, models, transforms
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using GPU!")
else:
    print("WARNING: Could not find GPU! Using CPU only")

PyTorch Version:  1.2.0
Torchvision Version:  0.4.0a0+6b959ee
Using GPU!


In [11]:
def make_CNN(model_name, num_classes, resume_from = None):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    # The model (nn.Module) to return
    model_ft = None
    # The input image is expected to be (input_size, input_size)
    input_size = 0
    
    # You may NOT use pretrained models!! 
    use_pretrained = False
    
    # By default, all parameters will be trained (useful when you're starting from scratch)
    # Within this function you can set .requires_grad = False for various parameters, if you
    # don't want to learn them

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224
        
    elif model_name == "vgg13":
        """ VGG13_bn
        """
        model_ft = custom_models.vgg13_bn(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224
        
    elif model_name == "vgg16":
        """ VGG16_bn
        """
        model_ft = custom_models.vgg13_bn(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224
        
    elif model_name == "vgg19":
        """ VGG19_bn
        """
        model_ft = custom_models.vgg13_bn(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes) 
        input_size = 224

    else:
        raise Exception("Invalid model name!")
    
    return model_ft, input_size

In [3]:
class Chest_Disease_Net(nn.Module):
    """
    fc1: number of neurons in the hidden fully connected layer
    """
    def __init__(self, cnn_model_name, num_classes, num_multimodal_features=12, fc1_out=32, resume_from=None):
        #num_classes = 14
        #num_multimodal_features= 12
        super(Chest_Disease_Net, self).__init__()
        self.cnn, self.input_size = make_CNN(cnn_model_name, num_classes)#models.vgg11(pretrained=False, progress = True)
        #define output layers
        self.fc1 = nn.Linear(num_classes + num_multimodal_features, fc1_out) #takes in input of CNN and multimodal input
        self.fc2 = nn.Linear(fc1_out, num_classes)
        if resume_from is not None:
            print("Loading weights from %s" % resume_from)
            self.load_state_dict(torch.load(resume_from))
        
    def forward(self, image, data):
        x1 = self.cnn(image)
        x2 = data 
        x = torch.cat((x1.float(), x2.float()), dim=1) ### ???
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x.double() ### ???

In [4]:
class MultimodalDataset(Dataset):
    """
    Custom dataset definition
    """
    def __init__(self, pandas_dataframe, img_path, transform=None):
        """
        """
        self.df = pandas_dataframe
        self.img_path = img_path
        self.transform = transform
            
    def __getitem__(self, index):
        """
        """
        img_name = self.df.iloc[index]["img_name"] 
        #print(img_name)
        img_path = os.path.join(self.img_path, img_name)
        image = Image.open(img_path)
        image = image.convert("RGB")
        image = np.asarray(image)
        #image = io.imread(image, as_gray = True)

        #print("Image shape: ", image.shape)
        if self.transform is not None:
            image = self.transform(image)
        dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor # ???
        features = np.fromstring(self.df.iloc[index]["feature"][1:-1], sep=",") # ???
        features = torch.from_numpy(features.astype("float")) # ???
        label = int(self.df.iloc[index]['label'])
        
        #print("Label type: ", type(label))
        #label = np.int_(label) #???
        #print("label type post casting: ", type(label))
        return image, features, label
        
    def __len__(self):
        return len(self.df)

In [5]:
def get_dataloaders(provided_df, input_size, batch_size):
    # How to transform the image when you are loading them.
    # you'll likely want to mess with the transforms on the training set.
    
    # For now, we resize/crop the image to the correct input size for our network,
    # then convert it to a [C,H,W] tensor, then normalize it to values with a given mean/stdev. These normalization constants
    # are derived from aggregating lots of data and happen to produce better results.
    data_transforms = {
        'test': transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.225])
        ])
    }
    # Create training and validation datasets
    data_set = MultimodalDataset(pandas_dataframe = provided_df, 
                                         img_path="/storage/images", 
                                         transform=data_transforms["test"])
    dataloaders_dict = DataLoader(data_set, batch_size=batch_size, shuffle=False, num_workers=4)
    return dataloaders_dict

In [6]:
def load_model(path):
    model_name = "vgg"
    num_classes = 14
    model = Chest_Disease_Net(cnn_model_name = model_name, num_classes = num_classes, resume_from = None)
    model_path = path
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint)
    return model

In [19]:
def per_disease(path): 
    df = pd.read_csv("./data/test_dataset.csv").drop(['Unnamed: 0', 'Unnamed: 0.1', 'dataset_type', 'disease'],axis=1) #test_dataset

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        
    disease_names = ['pneumothorax', 'pneumonia', 'pleuralthickening', 'nofinding', 'nodule', 'mass', 'infiltration', 'fibrosis', 'emphysema', 'effusion', 'edema', 'consolidation', 'cardiomegaly', 'atelectasis']
    disease_to_label = {}
    for index,value in enumerate(disease_names):
        disease_to_label[index] = value

      
    input_size = 256
    batch_size = 64
    
    
    model = load_model(path)    
    model.to(device)
    model.eval()
    
    for i in range(14):
        new_df = df[df["label"] == float(i)]
        dataloader = get_dataloaders(new_df, input_size, batch_size)
        correct = 0
        test_total = 0

        with torch.no_grad():
            for data in tqdm(dataloader):
                images, features, labels = data
                images = images.to(device)
                features = feature.to(device)
                labels = labels.to(device)

                outputs = model(images, features)
                _, predicted = torch.max(outputs.data, 1)

                test_total += labels.size(0)

                correct += (predicted == labels).sum().item()

        print('Top One Error of the network on test images for' + str(disease_to_label[i]) + ': %d %%' % (
                100 * (1 - correct / test_total)))
        

In [20]:
file_path = "/home/ubuntu/6.867-xray-project/weights/model.0"
per_disease(file_path)

RuntimeError: Error(s) in loading state_dict for Chest_Disease_Net:
	Missing key(s) in state_dict: "cnn.features.0.weight", "cnn.features.0.bias", "cnn.features.1.weight", "cnn.features.1.bias", "cnn.features.1.running_mean", "cnn.features.1.running_var", "cnn.features.4.weight", "cnn.features.4.bias", "cnn.features.5.weight", "cnn.features.5.bias", "cnn.features.5.running_mean", "cnn.features.5.running_var", "cnn.features.8.weight", "cnn.features.8.bias", "cnn.features.9.weight", "cnn.features.9.bias", "cnn.features.9.running_mean", "cnn.features.9.running_var", "cnn.features.11.weight", "cnn.features.11.bias", "cnn.features.12.weight", "cnn.features.12.bias", "cnn.features.12.running_mean", "cnn.features.12.running_var", "cnn.features.15.weight", "cnn.features.15.bias", "cnn.features.16.weight", "cnn.features.16.bias", "cnn.features.16.running_mean", "cnn.features.16.running_var", "cnn.features.18.weight", "cnn.features.18.bias", "cnn.features.19.weight", "cnn.features.19.bias", "cnn

In [21]:
checkpoint = torch.load("/home/ubuntu/6.867-xray-project/weights/model.0")


In [24]:
print(checkpoint.keys())

odict_keys(['layer1.0.weight', 'layer1.0.bias', 'layer2.0.weight', 'layer2.0.bias', 'layer3.0.weight', 'layer3.0.bias', 'layer4.0.weight', 'layer4.0.bias', 'layer5.0.weight', 'layer5.0.bias', 'fc1.0.weight', 'fc1.0.bias', 'fc2.0.weight', 'fc2.0.bias', 'fc3.weight', 'fc3.bias'])
